In [16]:
import pandas as pd
def filter_data():
    loc_df = pd.read_csv('data/전국공공시설개방정보표준데이터.csv', encoding='cp949')
    loc_df = loc_df[['관리기관명','담당부서명','위도','경도']]
    loc_df = loc_df[loc_df['관리기관명'].str.contains('서울특별시')]
    loc_df = loc_df.dropna()
    loc_df = loc_df[loc_df['담당부서명'].str.contains('주민센터')]
    loc_df = loc_df.drop_duplicates()
    return loc_df
loc_df = filter_data()

,관리기관명,담당부서명,위도,경도
0,서울특별시 노원구청,중계4동주민센터,37.658728,127.077958
1,서울특별시 노원구청,상계1동 주민센터,37.679930,127.054976
2,서울특별시 노원구청,상계1동주민센터,37.679930,127.054976
225,서울특별시 도봉구청,쌍문2동주민센터,37.658021,127.038751
228,서울특별시 도봉구청,쌍문4동주민센터,37.656419,127.028369
...,...,...,...,...
6204,서울특별시 영등포구청,도림동주민센터,37.509951,126.908024
6323,서울특별시 영등포구청,신길4동주민센터,37.508472,126.911276
6324,서울특별시 영등포구청,신길3동주민센터,37.507269,126.907744
6326,서울특별시 영등포구청,신길1동주민센터,37.511118,126.921433


In [ ]:
# STMO-P7
###fsfasf 
def a_star(graph, source, sink, heuristic):
    """
    Implementation of the A* shortest path algorithm
    Inputs:
        - graph : dict representing the graph
        - source : the source node
        - sink : the sink node (optional)
        - heuristic : a function with the heuristic for the shortest path between two nodes
    Ouput:
        - distance : dict with the distances of the nodes to the source
        - came_from : dict with for each node the previous node in the shortest
                    path from the source
    """
    # keep tentative distance source to vertex
    # initialize with infinity, except for the source
    distance = {v : inf for v in graph.keys()}
    distance[source] = 0
    # keep previous node in path for backtracking
    previous = {}
    # vertices_to_check is a heap using the estimated distance
    # of a given node to a source as the priority
    vertices_to_check = [(heuristic(source, sink), source)]             #complete
    previous = {}
    N_vertices = 0

#complete below (you can start from the Dijkstra algorithm)    
    while vertices_to_check:
        heuristic_dis, current = heappop(vertices_to_check)
        if current == sink:
            return reconstruct_path(previous, source, sink), distance[sink]
        for dist_current_neighbor, neighbor in graph[current]:
            new_dist_from_source = distance[current] + dist_current_neighbor
            if new_dist_from_source < distance[neighbor]:
                min_dist_neighbor_source = distance[neighbor] + heuristic(neighbor, sink)
                heappush(vertices_to_check, min_dist_neighbor_source, neighbor)
                previous[neighbor] = current

In [ ]:
# Example
graph = {
    'A': {'B': {'travelTime': 10, 'trafficTime': 2}, 'C': {'travelTime': 5, 'trafficTime': 1}},
    'B': {'C': {'travelTime': 8, 'trafficTime': 3}},
    'C': {'D': {'travelTime': 6, 'trafficTime': 2}},
    'D': {'E': {'travelTime': 4, 'trafficTime': 1}},
    'E': {}
}

def AStarWithTraffic(graph, start, goal, traffic_factor):
    openSet = []
    openSet.append((0, start))
    cameFrom = {}
    gScore = {}
    gScore[start] = 0
    fScore = {}
    fScore[start] = heuristic(start, goal)

    while openSet:
        current = min(openSet, key=lambda x: fScore[x[1]])
        openSet.remove(current)

        if current[1] == goal:
            return reconstructPath(cameFrom, current[1])

        for neighbor in graph.neighbors(current[1]):
            edge = graph.get_edge(current[1], neighbor)
            travelTime = edge.travelTime
            trafficTime = edge.trafficTime
            # Assume trafficTime is the additional time due to traffic conditions

            # Calculate the tentative gScore and fScore for the neighbor
            tentative_gScore = gScore[current[1]] + (travelTime + traffic_factor * trafficTime)
            tentative_fScore = tentative_gScore + heuristic(neighbor, goal)

            if neighbor not in gScore or tentative_gScore < gScore[neighbor]:
                cameFrom[neighbor] = current[1]
                gScore[neighbor] = tentative_gScore
                fScore[neighbor] = tentative_fScore

                if neighbor not in [node[1] for node in openSet]:
                    openSet.append((tentative_fScore, neighbor))

    return None  # No path found


'''In this updated version, we assume that the edge object has a new attribute trafficTime, which represents the additional time required due to traffic conditions along that edge. The traffic_factor is a parameter that you can adjust to control the influence of traffic on the estimated travel time.

The tentative gScore is calculated by adding the base travel time (travelTime) and the traffic-adjusted time (traffic_factor * trafficTime) to the previous gScore. This ensures that the algorithm considers the traffic conditions when determining the shortest path.

Please note that the exact implementation may depend on the specific structure of your graph and the representation of edge data. Adjust the code accordingly based on your graph implementation.
'''